# ABS Quarterly Liiving Cost Index 6467

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import matplotlib.pyplot as plt
import pandas as pd

# local imports
from abs_data_capture import (
    find_id,
    get_abs_meta_and_data,
    get_fs_constants,
    metacol,
    get_plot_constants,
)
from plotting import clear_chart_dir, line_plot, plot_growth_finalise, set_chart_dir

# pandas display settings
pd.options.display.max_rows = 999999
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

# display charts in this notebook
SHOW = False

## Get data from ABS

NOTE: A bit ugly - the ABS does not appear to provide a complete summary zip-file for this one.

In [2]:
catalogue_id = "6467"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)
abs_dict = get_abs_meta_and_data(catalogue_id)
meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)

Found URL for a ZIP file on ABS web page
Retrieving data from cache: ABS_CACHE/d21e6cb83c6918fdc03644f5b3296b6a--Time-Series-Spreadsheets-all.zip
Extracting DataFrames from the zip-file ...


## Plot

### Manage chart directory

In [3]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)
plt.style.use("fivethirtyeight")

### Headline LCI for Australia

In [4]:
table = "1"
data = abs_dict[table]

# Used for pattern matching retrieval
quarterly_phrase = "Percentage Change from Previous Period"
annual_phrase = "Percentage Change from Corresponding Quarter of Previous Year"

labels = [
    x[1].strip()
    for x in meta[
        meta[metacol.did].str.contains(annual_phrase) & (meta[metacol.table] == table)
    ][metacol.did].str.split(";")
]

annuals = {}
for label in labels:
    a_id, _ = find_id(
        meta,
        {
            table: metacol.table,
            annual_phrase: metacol.did,
            label: metacol.did,
        },
    )
    annuals[label] = data[a_id]
    q_id, _ = find_id(
        meta,
        {
            table: metacol.table,
            quarterly_phrase: metacol.did,
            label: metacol.did,
        },
    )
    MAX_LEN = 35
    sep = "\n" if len(label) > MAX_LEN else " "
    for plot_from, plot_tag in zip(plot_times, plot_tags):
        p_plot_from = (
            None if plot_from is None else pd.Period(plot_from, freq=data.index.freq)
        )
        plot_growth_finalise(
            data[a_id],
            data[q_id],
            from_=p_plot_from,
            title=f"Living Cost Index:{sep}{label}",
            tag=plot_tag,
            axhspan={
                "ymin": 2,
                "ymax": 3,
                "color": "#dddddd",
                "label": "2-3% inflation target",
                "zorder": -1,
            },
            rfooter=f"{source} {table}",
            lfooter=f"Australia. Right-side annual rate: {data[a_id].iloc[-1]}%",
            show=SHOW,
        )

line_plot(
    pd.DataFrame(annuals),
    title=f"Annual Growth in Living Cost Indexes",
    ylabel="Per cent",
    axhspan={
        "ymin": 2,
        "ymax": 3,
        "color": "#dddddd",
        "label": "2-3% inflation target",
        "zorder": -1,
    },
    legend={"fontsize": "xx-small", "ncol": 2},
    rfooter=f"{source} {table}",
    lfooter="Australia. ",
    show=SHOW,
)

### LCI components

In [5]:
table = "3"
data = abs_dict[table]

typelabels = set(
    [
        x[2].strip()
        for x in meta[
            meta[metacol.did].str.contains(annual_phrase) & (meta[metacol.table] == table)
        ][metacol.did].str.split(";")
    ]
)

for typelabel in typelabels:
    annuals = {}
    for label in labels:
        a_id, _ = find_id(
            meta,
            {
                table: metacol.table,
                annual_phrase: metacol.did,
                label: metacol.did,
                typelabel: metacol.did,
            },
            verbose=True,
        )
        annuals[label] = data[a_id]

    line_plot(
        pd.DataFrame(annuals),
        title=f"LCI Annual Growth: {typelabel}",
        ylabel="Per cent",
        axhspan={
            "ymin": 2,
            "ymax": 3,
            "color": "#dddddd",
            "label": "2-3% inflation target",
            "zorder": -1,
        },
        legend={"fontsize": "xx-small", "ncol": 2},
        rfooter=f"{source} {table}",
        lfooter="Australia. ",
        show=SHOW,
    )

## Finished

In [6]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sun Feb 18 2024 12:32:54

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.21.0

matplotlib: 3.8.3
sys       : 3.11.8 | packaged by conda-forge | (main, Feb 16 2024, 20:49:36) [Clang 16.0.6 ]
pandas    : 2.2.0

Watermark: 2.4.3



In [7]:
print("Finished")

Finished
